# Introduction and Data Collection

## Scope of the Project

The scope of this project is to apply data science tools to assess the fuel consumption of internal combustion engine cars, based on a limited amount of information. In addition to standard data analysis, unsupervised learning will be applied to identify groups of vehicles that share similar characteristics, and supervised learning will be performed to train a predictor for the fuel consumption variable.

While data on fuel consumption is widely available to the consumer for any given car model, due to the very large amount of different models and engines that the market has to offer, it is likely that such customer will want to first limit the scope of his search to some class of vehicles (for instance, compact cars with a midsize engine). Doing so, however, requires to have at least a basic understanding of the characteristics of such class, and for sure fuel consumption is among the most significant ones. Hence, there is some value in developing models that predict fuel consumption based on a limited number of variables - those variables that define the broad class in which the consumer begins his search. Moreover, these models could also be used to quickly evaluate the fuel consumption of a vehicle when Internet access is unavailable or impractical.

The present notebook deals with retrieving the data from the source, making it available for the subsequent steps of the work. A subset of the original data is extracted with no modifications, and stored in Parquet format to preserve the original data types. Later, appropriate transformations will make the data ready for analysis and machine learning.

In [1]:
import urllib
import json

import pandas as pd

## Data Source

The data we will use for this project is sourced from the [European Environment Agency Datahub](https://www.eea.europa.eu/en/datahub). According to their [legal notice](https://www.eea.europa.eu/en/legal-notice), the EEA databases are published under the [CC-BY](https://creativecommons.org/licenses/by/4.0/) license. We will download part of the data regarding the [Monitoring of CO2 emissions from passenger cars](https://www.eea.europa.eu/en/datahub/datahubitem-view/fa8b1229-3db6-495d-b18e-9c9b3267c02b). As stated in the official page:

> The Regulation (EU) No 2019/631 requires Countries to record information for each new passenger car registered in its territory. Every year, each Member State shall submit to the Commission all the information related to their new registrations. In particular, the following details are required for each new passenger car registered: manufacturer name, type approval number, type, variant, version, make and commercial name, specific emissions of CO2 (NEDC and WLTP protocols), masses of the vehicle, wheel base, track width, engine capacity and power, fuel type and mode, eco-innovations and electricity consumption. Data for EU-27 and UK are reported in the main database.

The size of the data provided is very large (in the order of tens of gigabytes), hence it is impractical to download it directly from the website. However, the EEA offers a great tool to query their databases, the [Discodata](https://discodata.eea.europa.eu/) service. In addition to a web application to directly inspect the data, Discodata provides an API to retrieve data in JSON format by making SQL queries. Using such API, we are able to precisely select the data we need and download it efficiently, while at the same time providing the necessary code for reproducibility.

We follow the [Discodata documentation](https://discodata.eea.europa.eu/Help.html) to construct the appropriate URLs for querying the database, and to correctly interpret the JSON response.

In [2]:
def query_discodata(sql_query, page=1, max_results=100):
    url_query = {
        "query": sql_query,
        "p": page,
        "nrOfHits": max_results
    }
    url = f"https://discodata.eea.europa.eu/sql?{urllib.parse.urlencode(url_query)}"

    with urllib.request.urlopen(url) as f:
        response = json.load(f)

    if "results" in response:
        return response["results"]
    else:
        raise Exception(f"unable to execute the query: {response}")

Let's test the above function by retrieving the first car. The table of our interest is `co2cars`. For future reproducibility, we explicitly specify a version and revision number, which at the present time correspond to the latest revision of the data regarding new vehicle registrations in the year 2023.

In [3]:
table_ref = "[CO2Emission].[v7r1].[co2cars_2023Fv28]"
query_discodata(f"SELECT TOP 1 * FROM {table_ref}")

[{'ID': 128825420,
  'MS': 'IT',
  'Mp': 'STELLANTIS',
  'VFN': 'IP-03_312_0344-ZFA-1',
  'Mh': 'STELLANTIS EUROPE',
  'Man': 'STELLANTIS EUROPE SPA',
  'MMS': None,
  'TAN': 'E3*2007/46*0064*69',
  'T': '312',
  'Va': 'PYD1B',
  'Ve': 'AA4',
  'Mk': 'FIAT',
  'Cn': 'PANDA',
  'Ct': 'M1',
  'Cr': 'M1',
  'M (kg)': 1045,
  'Mt': 1114,
  'Enedc (g/km)': None,
  'Ewltp (g/km)': 109,
  'W (mm)': None,
  'At1 (mm)': None,
  'At2 (mm)': None,
  'Ft': 'petrol',
  'Fm': 'H',
  'Ec (cm3)': 999,
  'Ep (KW)': 52,
  'Z (Wh/km)': None,
  'IT': 'e3 33',
  'Ernedc (g/km)': None,
  'Erwltp (g/km)': 1.28,
  'De': None,
  'Vf': None,
  'R': 1,
  'Year': 2023,
  'Status': 'F',
  'Version_file': 'v28',
  'E (g/km)': None,
  'Er (g/km)': None,
  'Zr': None,
  'Dr': '2023-09-19',
  'Fc': 4.8,
  'Ech': '',
  'RLFI': ''}]

There is clearly a large number of columns here, most of which are not interesting for our purposes. The EEA dataset website also provides a [Table definition.xlsx](https://sdi.eea.europa.eu/catalogue/srv/api/records/87fd2bce-6ad5-46d8-af41-f27cfd2e45a8/attachments/Table-definition.xlsx) file as an attachment. Let's load it here in a Pandas DataFrame.

In [4]:
url = "https://sdi.eea.europa.eu/catalogue/srv/api/records/87fd2bce-6ad5-46d8-af41-f27cfd2e45a8/attachments/Table-definition.xlsx"
pd.read_excel(url, skiprows=2)

,Name,Definition,Datatype,Cardinality
0,ID,Identification number.,integer,1..1
1,MS,Member state.,varchar(2),0..1
2,Mp,Manufacturer pooling.,varchar(50),0..1
3,VFN,Vehicle family identification number.,varchar(25),0..1
4,Mh,Manufacturer name EU standard denomination .,varchar(50),0..1
5,Man,Manufacturer name OEM declaration.,varchar(50),0..1
6,MMS,Manufacturer name MS registry denomination .,varchar(125),0..1
7,TAN,Type approval number.,varchar(50),0..1
8,T,Type.,varchar(25),0..1
9,Va,Variant.,varchar(25),0..1


According to the [guidelines of the European Commission](https://circabc.europa.eu/sd/a/d9cff59f-5117-48f4-9a37-07b94027110c/MS%20Guidelines%202019), "M1" is the vehicle category representing passenger cars, while the fuel mode is classified as follows:

|Code|Description|
|---|---|
|M|Mono-fuel vehicles, i.e. vehicles able to run on only one fuel, either petrol, diesel, LPG, natural gas (NG) or hydrogen.|
|B|Bi-fuel vehicles, i.e. vehicles with two separate fuel storage systems, which are designed to run primarily on only one fuel at a time. This covers vehicles that can run on petrol and either LPG, NG/biomethane or hydrogen.|
|F|Flex-fuel vehicles, i.e. vehicles with one fuel storage system that can run on different mixtures of two or more fuels; this concerns more specifically "flex fuel ethanol vehicles", which can run on petrol or a mixture of petrol and ethanol up to an 85 per cent ethanol blend (E85).|
|E|Battery electric vehicles (BEV), i.e. "pure" electric vehicles (NOT hybrid vehicles).|
|P|Off vehicle charging hybrid electric vehicles (OVC-HEV), i.e. plug-in hybrid vehicles.|
|H|Not-Off vehicle charging hybrid electric vehicles (NOVC-HEV). These vehicles cannot take electric energy from external sources.|

In [5]:
query_discodata(f"""
    SELECT DISTINCT Ct, count(*) AS cnt
    FROM {table_ref}
    GROUP BY Ct
""")

[{'Ct': '', 'cnt': 13212},
 {'Ct': 'M1', 'cnt': 10334971},
 {'Ct': 'M1G', 'cnt': 385951},
 {'Ct': 'N1', 'cnt': 684},
 {'Ct': 'N1G', 'cnt': 28},
 {'Ct': 'N2', 'cnt': 51},
 {'Ct': 'N2G', 'cnt': 1}]

In [6]:
query_discodata(f"""
    SELECT DISTINCT Fm, Ft, count(*) AS cnt
    FROM {table_ref}
    GROUP BY Fm, Ft
    ORDER BY Fm, Ft
""")

[{'Fm': 'B', 'Ft': 'lpg', 'cnt': 292584},
 {'Fm': 'B', 'Ft': 'ng', 'cnt': 2490},
 {'Fm': 'E', 'Ft': 'electric', 'cnt': 1669222},
 {'Fm': 'F', 'Ft': 'e85', 'cnt': 7357},
 {'Fm': 'H', 'Ft': 'diesel', 'cnt': 453878},
 {'Fm': 'H', 'Ft': 'e85', 'cnt': 38849},
 {'Fm': 'H', 'Ft': 'petrol', 'cnt': 2362321},
 {'Fm': 'M', 'Ft': 'diesel', 'cnt': 1369629},
 {'Fm': 'M', 'Ft': 'hydrogen', 'cnt': 730},
 {'Fm': 'M', 'Ft': 'lpg', 'cnt': 105},
 {'Fm': 'M', 'Ft': 'ng', 'cnt': 4875},
 {'Fm': 'M', 'Ft': 'petrol', 'cnt': 3692854},
 {'Fm': 'P', 'Ft': 'diesel/electric', 'cnt': 30304},
 {'Fm': 'P', 'Ft': 'petrol/electric', 'cnt': 809700}]

We focus our analysis on passenger cars, and since we are interested in fuel consumption, we exclude electric cars. For simplicity, we also exclude hybrid cars, as it would probably make sense to conduct an entirely separate analysis for this class.

We discard the columns regarding manufacturing details, as well as columns containing codes required for car registration. While it will surely be correlated with fuel consumption, we also discard data on CO2 emissions, because it is improbable that one has access to such data without also having access to the fuel consumption data itself. Furthermore, as stated at the beginning of this notebook, the main aim of this project is to be able to predict fuel consumption based on a limited set of simple variables, and CO2 emissions doesn't seem to fit among those.

Finally, we also discard columns that are always NULL (wheel base, axle widths, deviation factor, and verification factor) or constant (number of registrations).

In [7]:
query_discodata(f"SELECT DISTINCT [W (mm)], [At1 (mm)], [At2 (mm)], De, Vf, R FROM {table_ref}")

[{'W (mm)': None,
  'At1 (mm)': None,
  'At2 (mm)': None,
  'De': None,
  'Vf': None,
  'R': 1}]

We now construct a dictionary in which we indicate the columns that we want to extract, and a meaningful name with which we want to rename them. Since JSON is a typed data format, we don't expect an explicit data type conversion to be required.

Note that we are retaining the column "ID", hence all the vehicle registrations of the year 2023 are kept. Of course, this means that each car configuration will be present multiple times in our data. The logic behind this choice is that cars that are registered more often are more likely to be chosen by the customers, hence they should have more weight in the evaluation metric used for predictive modeling. Or, in probabilistic terms, by keeping all vehicle registrations, we make sure that the probability space from which the features are sampled coincides with the probability space from which customers draw their choices, ensuring the reliability of our metric.

In [8]:
final_cols = {
    "ID": "id",
    "Mp": "manufacturer",
    "Mk": "brand",
    "Cn": "model",
    "M (kg)": "mass_kg",
    "Ft": "fuel_type",
    "Fm": "fuel_mode",
    "Ec (cm3)": "engine_size_cm3",
    "Ep (KW)": "engine_power_kw",
    "Fc": "fuel_cons_l_100km"
}

In [9]:
final_query = f"""
    SELECT {", ".join(f"[{col}] AS {name}" for col, name in final_cols.items())}
    FROM {table_ref}
    WHERE Ct = 'M1' AND Cr = 'M1' AND Fm IN ('M', 'B', 'F') AND Fc IS NOT NULL
"""
query_discodata(final_query, max_results=1)

[{'id': 9,
  'manufacturer': 'RENAULT-NISSAN-MITSUBISHI',
  'brand': 'MITSUBISHI MOTORS THAILAND',
  'model': 'MITSUBISHI SPACE STAR',
  'mass_kg': 940,
  'fuel_type': 'petrol',
  'fuel_mode': 'M',
  'engine_size_cm3': 1193,
  'engine_power_kw': 52,
  'fuel_cons_l_100km': 5.0}]

## Data Extraction

Now that we have chosen the columns and rows that we are interested in, we can proceed with the extraction. We do not perform any preprocessing transformations at this level, so that:

- we ensure the reproducibility of the results;
- we allow for different preprocessing actions for different machine learning models (for instance, some imputation method might work well for a model and badly for another);
- we avoid the risk of compromising the data before performing a train-test split.

First, we want to estimate whether the whole data fits in memory. If that's not the case, we will need to write to disk multiple files while we cycle through the API requests.

In [10]:
query_discodata(f"SELECT count(*) AS cnt FROM ({final_query}) AS subquery")

[{'cnt': 5152489}]

In [11]:
pd.DataFrame(query_discodata(final_query, max_results=100)).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 100 non-null    int64  
 1   manufacturer       100 non-null    object 
 2   brand              100 non-null    object 
 3   model              100 non-null    object 
 4   mass_kg            100 non-null    int64  
 5   fuel_type          100 non-null    object 
 6   fuel_mode          100 non-null    object 
 7   engine_size_cm3    100 non-null    int64  
 8   engine_power_kw    100 non-null    int64  
 9   fuel_cons_l_100km  100 non-null    float64
dtypes: float64(1), int64(4), object(5)
memory usage: 7.9+ KB


Since a DataFrame constructed with a sample of 100 records uses less than 8 KB of memory, and there are approximately 5 million records to extract, we estimate that a DataFrame containing the entire data will use approximately 400 MB of memory. We can therefore load the entire data in memory, and only at then end will we write it to disk as a single Parquet file.

According to the Discodata documentation, we can end the cycle when the retrieved results list is empty.

In [12]:
data_frames = []
page = 1
while results := query_discodata(final_query, page, max_results=10000):
    data_frames.append(pd.DataFrame(results))
    page += 1

In [13]:
source_data = pd.concat(data_frames).set_index('id')
display(source_data.sample(10))
source_data.info()

,manufacturer,brand,model,mass_kg,fuel_type,fuel_mode,engine_size_cm3,engine_power_kw,fuel_cons_l_100km
id,,,,,,,,,
122251551,STELLANTIS,PEUGEOT,208,1165.0,petrol,M,1199.0,74.0,5.1
122593150,RENAULT-NISSAN-MITSUBISHI,DACIA,SANDERO,1090.0,petrol,M,999.0,49.0,5.3
130846724,VOLKSWAGEN,SKODA,OCTAVIA,1499.0,diesel,M,1968.0,110.0,4.5
132626607,KIA,KIA,SPORTAGE,1526.0,petrol,M,1598.0,110.0,6.7
124134362,VOLKSWAGEN,VOLKSWAGEN,T-CROSS,1290.0,petrol,M,999.0,81.0,6.2
133858186,STELLANTIS,OPEL,CORSA,1055.0,petrol,M,1199.0,55.0,5.3
127427250,VOLKSWAGEN,SKODA,KAMIQ,1301.0,petrol,M,1498.0,110.0,6.0
127399168,VOLKSWAGEN,VOLKSWAGEN,T-ROC,1358.0,petrol,M,1498.0,110.0,6.4
133541298,KIA,KIA,RIO,1120.0,petrol,M,1197.0,62.0,5.3


<class 'pandas.core.frame.DataFrame'>
Index: 5152489 entries, 9 to 140000054
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   manufacturer       object 
 1   brand              object 
 2   model              object 
 3   mass_kg            float64
 4   fuel_type          object 
 5   fuel_mode          object 
 6   engine_size_cm3    float64
 7   engine_power_kw    float64
 8   fuel_cons_l_100km  float64
dtypes: float64(4), object(5)
memory usage: 393.1+ MB


We are happy with the result and we can write it to disk. We choose the Parquet format, not only to take advantage of compression, but also to retain the original data types. Since we will process the data in-memory, we decide to favor compression ratio rather than compression speed, choosing the Brotli algorithm. The next notebook will perform some data analysis and (if necessary) data cleaning.

In [14]:
source_data.to_parquet("../data/source.parquet", compression='brotli')